In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [23]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')

In [4]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [5]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-train_user_id_min,err-1] +=1
error

100%|██████████| 16554663/16554663 [00:48<00:00, 343839.72it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [6]:
problem = np.zeros(15000)
problem[train_problem['user_id'].unique()-train_user_id_min] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [7]:
train_x = error
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 42)
(15000,)


In [18]:
dtrain = xgb.DMatrix(data=train_x,label=train_y)

In [8]:
def f_pr_auc(probas_pred,y_true):
  labels = y_true.get_label()
  p, r, _ = precision_recall_curve(labels,probas_pred)
  score = auc(r,p)
  return 'pr_auc',score,True

In [13]:
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5

# 파라미터 설정
params =      {
                'objective'     : 'binary:logistic',
                'eval_metric'        : 'auc',
               'early_stoppings'  :   100
                }
xgb_model = xgb.train(params=params,
                      dtrain=dtrain,
                      feval=f_pr_auc,
                      num_boost_round=1000)


In [14]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [15]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
  test_x[person_idx-test_user_id_min,err-1]+=1
test_x = test_x.reshape(test_x.shape[0],-1)
test_x.shape

100%|██████████| 16532648/16532648 [00:49<00:00, 337309.72it/s]


(14999, 42)

In [19]:
dtest = xgb.DMatrix(data=test_x)

In [22]:
pred_y = xgb_model.predict(dtest)
pred = np.where(pred_y>threshold,1,0)
pred

array([1, 0, 0, ..., 1, 1, 1])

In [24]:
sample_submission.head(2)

,user_id,problem
0,30000,0
1,30001,0


In [26]:
sample_submission['problem'] = pred
sample_submission.head()

,user_id,problem
0,30000,1
1,30001,0
2,30002,0
3,30003,1
4,30004,1


In [27]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/xgboost_submission.csv')